# AUSTIN MONTE - API DATA REPORT - 10/11/24
### HYPOTHESIS - Tracking popularity, energy, and tempo, I plan to use those values to determine which of Bruno Mars' studio albums is the favorite by the public. As a critically acclaimed artist with multiple awards, all of his albums are highly praised and being able to use these statistics from Spotify's API will help solve my hypothesis.

#### Majority of Bruno Mars' music is upbeat, pop music that does well on the Billboard top 100, so that is why I believe that tracking popularity, energy, and tempo will be the best data to use for my hypothesis.

#### Bruno Mars has been making music since the early 2010s, before Spotify's existence. Because of this, for his earlier albums, the popularity data might not be as accurate as his later albums, since his earlier albums were mostly sold on CDs instead of streaming.

In [35]:
import urllib
import requests
import json
import pandas as pd
import base64

In [36]:
def get_session_token(SessionID, SessionKey):
    url = 'https://accounts.spotify.com/api/token'
    data = {'grant_type':'client_credentials'}
    encoded_key = base64.b64encode(str(SessionID + ":" + SessionKey).\
                                    encode("ascii"))
    header = {'Authorization': 'Basic {}'.format(encoded_key.decode("ascii"))}

    response = requests.post(url,
                            data = data,
                           headers = header)
    print(response.status_code)
    return response.json()['access_token']

In [37]:
keys = pd.read_csv("spotify api.txt")

In [38]:
access_token = get_session_token(keys['Client_ID'].iloc[0],keys['Client_Secret'].iloc[0])

200


In [39]:
t_features_ep = 'https://api.spotify.com/v1/audio-features'
tracks_ep = 'https://api.spotify.com/v1/tracks'
ab_tracks_ep = 'https://api.spotify.com/v1/albums/{}/tracks'

In [40]:
def api_call(endpoint_url, api_header):
    response = requests.get(endpoint_url, headers = api_header)
    print(response.status_code)
    return response.json()

In [41]:
session_header = {'Authorization': 'Bearer {}'.format(access_token)}

In [42]:
dw_album_id = "1uyf3l2d4XYwiEqAb7t7fX"
#Doo-Woops & Hooligans
uj_album_id = "58ufpQsJ1DS5kq4hhzQDiI"
#Unorthodox Jukebox
tf_album_id = "4PgleR09JVnm3zY1fW3XBA"
#24K Magic
ss_album_id = "4VZ7jhV0wHpoNPCB7Vmiml"
#An Evening with Silk Sonic

In [43]:
ab_tracks_ep.format(dw_album_id)
ab_tracks_ep.format(uj_album_id)
ab_tracks_ep.format(tf_album_id)
ab_tracks_ep.format(ss_album_id)

'https://api.spotify.com/v1/albums/4VZ7jhV0wHpoNPCB7Vmiml/tracks'

In [44]:
dw_album_response = api_call(ab_tracks_ep.format(dw_album_id), session_header)
uj_album_response = api_call(ab_tracks_ep.format(uj_album_id), session_header)
tf_album_response = api_call(ab_tracks_ep.format(tf_album_id), session_header)
ss_album_response = api_call(ab_tracks_ep.format(ss_album_id), session_header)

200
200
200
200


In [45]:
dw_album_df = pd.DataFrame(dw_album_response['items'])
uj_album_df = pd.DataFrame(uj_album_response['items'])
tf_album_df = pd.DataFrame(tf_album_response['items'])
ss_album_df = pd.DataFrame(ss_album_response['items'])

In [46]:
dw_track_ids = ','.join(dw_album_df['id'].to_list())
uj_track_ids = ','.join(uj_album_df['id'].to_list())
tf_track_ids = ','.join(tf_album_df['id'].to_list())
ss_track_ids = ','.join(ss_album_df['id'].to_list())

In [47]:
t_features_ep + '?ids={}'.format(dw_track_ids)
t_features_ep + '?ids={}'.format(uj_track_ids)
t_features_ep + '?ids={}'.format(tf_track_ids)
t_features_ep + '?ids={}'.format(ss_track_ids)

'https://api.spotify.com/v1/audio-features?ids=4K09vJ27xCOreumtSuU6Ao,4pryE6cN2gFL1FVF5fYINl,7suB6D6uKX5DfPukdGaz0W,6jGAh1bFnXt1Muj9zeHveZ,1oERlssLrpssCAY6Yqqs6c,5lka5RUbLVQGO94mKAPMRO,2K6vUau7bnZUamjbRSOOvJ,3WTWh2WDk4j8GUCGj4xfOd,2ALh2jqA7KldpHMUHvRomw,2NqyjfDXy0XfXCSPXMsKzi'

In [48]:
dw_track_features = api_call(t_features_ep + '?ids={}'.format(dw_track_ids), session_header)
uj_track_features = api_call(t_features_ep + '?ids={}'.format(uj_track_ids), session_header)
tf_track_features = api_call(t_features_ep + '?ids={}'.format(tf_track_ids), session_header)
ss_track_features = api_call(t_features_ep + '?ids={}'.format(ss_track_ids), session_header)

200
200
200
200


In [49]:
dw_track_info = api_call(tracks_ep + '?market=US&ids={}'.format(dw_track_ids), session_header)
uj_track_info = api_call(tracks_ep + '?market=US&ids={}'.format(uj_track_ids), session_header)
tf_track_info = api_call(tracks_ep + '?market=US&ids={}'.format(tf_track_ids), session_header)
ss_track_info = api_call(tracks_ep + '?market=US&ids={}'.format(ss_track_ids), session_header)

200
200
200
200


In [50]:
dw_features_df = pd.DataFrame(dw_track_features['audio_features'])
dw_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.704,0.558,2,-7.273,0,0.0542,0.1480,0.0,0.1070,0.245,110.444,audio_features,2tJulUYLDKOg9XrtVkMgcJ,spotify:track:2tJulUYLDKOg9XrtVkMgcJ,https://api.spotify.com/v1/tracks/2tJulUYLDKOg...,https://api.spotify.com/v1/audio-analysis/2tJu...,222091,4
1,0.635,0.841,5,-5.379,1,0.0422,0.0134,0.0,0.0622,0.424,109.021,audio_features,7BqBn9nzAq8spo5e7cZ0dJ,spotify:track:7BqBn9nzAq8spo5e7cZ0dJ,https://api.spotify.com/v1/tracks/7BqBn9nzAq8s...,https://api.spotify.com/v1/audio-analysis/7BqB...,220734,4
2,0.709,0.450,9,-8.267,0,0.0428,0.4190,0.0,0.2410,0.873,77.986,audio_features,5BmKRyAjCnvkko7Ms00YtA,spotify:track:5BmKRyAjCnvkko7Ms00YtA,https://api.spotify.com/v1/tracks/5BmKRyAjCnvk...,https://api.spotify.com/v1/audio-analysis/5BmK...,243004,4
3,0.658,0.762,10,-3.028,0,0.0425,0.2630,0.0,0.1270,0.823,163.864,audio_features,5MMLS3xm12D7N26xlfFApr,spotify:track:5MMLS3xm12D7N26xlfFApr,https://api.spotify.com/v1/tracks/5MMLS3xm12D7...,https://api.spotify.com/v1/audio-analysis/5MML...,148448,4
4,0.794,0.711,8,-5.124,0,0.0699,0.3000,0.0,0.0955,0.955,174.915,audio_features,1ExfPZEiahqhLyajhybFeS,spotify:track:1ExfPZEiahqhLyajhybFeS,https://api.spotify.com/v1/tracks/1ExfPZEiahqh...,https://api.spotify.com/v1/audio-analysis/1Exf...,189109,4


In [52]:
uj_features_df = pd.DataFrame(uj_track_features['audio_features'])
uj_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.561,0.749,6,-4.951,0,0.0320,0.4610,0.000002,0.2050,0.501,125.986,audio_features,3G5iN5QBqMeXx3uZPy8tgB,spotify:track:3G5iN5QBqMeXx3uZPy8tgB,https://api.spotify.com/v1/tracks/3G5iN5QBqMeX...,https://api.spotify.com/v1/audio-analysis/3G5i...,228720,4
1,0.726,0.698,5,-4.165,1,0.0431,0.0490,0.000000,0.3090,0.867,143.994,audio_features,3w3y8KPTfNeOKPiqUTakBh,spotify:track:3w3y8KPTfNeOKPiqUTakBh,https://api.spotify.com/v1/tracks/3w3y8KPTfNeO...,https://api.spotify.com/v1/audio-analysis/3w3y...,233478,4
2,0.717,0.544,7,-6.598,1,0.0371,0.4450,0.000005,0.0973,0.641,139.959,audio_features,2ih2U8ttFzCjnQ5njF3SrR,spotify:track:2ih2U8ttFzCjnQ5njF3SrR,https://api.spotify.com/v1/tracks/2ih2U8ttFzCj...,https://api.spotify.com/v1/audio-analysis/2ih2...,244493,4
3,0.874,0.692,5,-5.280,0,0.0431,0.0412,0.000072,0.3240,0.937,116.017,audio_features,55h7vJchibLdUkxdlX3fK7,spotify:track:55h7vJchibLdUkxdlX3fK7,https://api.spotify.com/v1/tracks/55h7vJchibLd...,https://api.spotify.com/v1/audio-analysis/55h7...,178560,4
4,0.705,0.784,2,-4.351,1,0.0279,0.2110,0.000004,0.0813,0.418,104.980,audio_features,30raivfq7rSt5nKltiHfzG,spotify:track:30raivfq7rSt5nKltiHfzG,https://api.spotify.com/v1/tracks/30raivfq7rSt...,https://api.spotify.com/v1/audio-analysis/30ra...,228573,4


In [53]:
tf_features_df = pd.DataFrame(tf_track_features['audio_features'])
tf_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.818,0.803,1,-4.282,1,0.0797,0.0340,0.000000,0.1530,0.632,106.970,audio_features,6b8Be6ljOzmkOmFslEb23P,spotify:track:6b8Be6ljOzmkOmFslEb23P,https://api.spotify.com/v1/tracks/6b8Be6ljOzmk...,https://api.spotify.com/v1/audio-analysis/6b8B...,225983,4
1,0.571,0.539,5,-4.446,0,0.1160,0.0208,0.000000,0.1250,0.727,202.103,audio_features,0mBKv9DkYfQHjdMcw2jdyI,spotify:track:0mBKv9DkYfQHjdMcw2jdyI,https://api.spotify.com/v1/tracks/0mBKv9DkYfQH...,https://api.spotify.com/v1/audio-analysis/0mBK...,186973,4
2,0.853,0.871,8,-3.715,1,0.0851,0.0145,0.000068,0.1980,0.863,124.021,audio_features,1I6pKIyaBp4OebTGLJpCCC,spotify:track:1I6pKIyaBp4OebTGLJpCCC,https://api.spotify.com/v1/tracks/1I6pKIyaBp4O...,https://api.spotify.com/v1/audio-analysis/1I6p...,210053,4
3,0.853,0.560,1,-4.961,1,0.0406,0.0130,0.000000,0.0944,0.860,134.066,audio_features,0KKkJNfGyhkQ5aFogxQAPU,spotify:track:0KKkJNfGyhkQ5aFogxQAPU,https://api.spotify.com/v1/tracks/0KKkJNfGyhkQ...,https://api.spotify.com/v1/audio-analysis/0KKk...,206693,4
4,0.578,0.574,2,-6.209,1,0.0454,0.1960,0.000000,0.0830,0.301,174.152,audio_features,0kN8xEmgMW9mh7UmDYHlJP,spotify:track:0kN8xEmgMW9mh7UmDYHlJP,https://api.spotify.com/v1/tracks/0kN8xEmgMW9m...,https://api.spotify.com/v1/audio-analysis/0kN8...,261240,4


In [54]:
ss_features_df = pd.DataFrame(ss_track_features['audio_features'])
ss_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.696,0.553,1,-8.702,1,0.0759,0.09700,0.000004,0.6170,0.674,114.993,audio_features,4K09vJ27xCOreumtSuU6Ao,spotify:track:4K09vJ27xCOreumtSuU6Ao,https://api.spotify.com/v1/tracks/4K09vJ27xCOr...,https://api.spotify.com/v1/audio-analysis/4K09...,63060,4
1,0.586,0.616,5,-7.964,1,0.0324,0.18200,0.000000,0.0927,0.719,148.088,audio_features,4pryE6cN2gFL1FVF5fYINl,spotify:track:4pryE6cN2gFL1FVF5fYINl,https://api.spotify.com/v1/tracks/4pryE6cN2gFL...,https://api.spotify.com/v1/audio-analysis/4pry...,242096,4
2,0.783,0.627,2,-8.521,1,0.1020,0.00756,0.000000,0.0606,0.480,100.025,audio_features,7suB6D6uKX5DfPukdGaz0W,spotify:track:7suB6D6uKX5DfPukdGaz0W,https://api.spotify.com/v1/tracks/7suB6D6uKX5D...,https://api.spotify.com/v1/audio-analysis/7suB...,219389,4
3,0.651,0.703,0,-8.958,0,0.0816,0.02970,0.000000,0.0608,0.647,140.051,audio_features,6jGAh1bFnXt1Muj9zeHveZ,spotify:track:6jGAh1bFnXt1Muj9zeHveZ,https://api.spotify.com/v1/tracks/6jGAh1bFnXt1...,https://api.spotify.com/v1/audio-analysis/6jGA...,249444,4
4,0.627,0.618,2,-8.529,1,0.0437,0.05580,0.000000,0.3510,0.848,82.030,audio_features,1oERlssLrpssCAY6Yqqs6c,spotify:track:1oERlssLrpssCAY6Yqqs6c,https://api.spotify.com/v1/tracks/1oERlssLrpss...,https://api.spotify.com/v1/audio-analysis/1oER...,197443,4


In [59]:
dw_tracks_df = pd.DataFrame(dw_track_info['tracks'])
uj_tracks_df = pd.DataFrame(uj_track_info['tracks'])
tf_tracks_df = pd.DataFrame(tf_track_info['tracks'])
ss_tracks_df = pd.DataFrame(ss_track_info['tracks'])

In [60]:
dw_merged = pd.merge(dw_features_df, dw_tracks_df,
                     how = 'inner', on = 'id')
dw_merged.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,external_urls,href,is_local,is_playable,name,popularity,preview_url,track_number,type_y,uri_y
0,0.704,0.558,2,-7.273,0,0.0542,0.1480,0.0,0.1070,0.245,...,{'spotify': 'https://open.spotify.com/track/2t...,https://api.spotify.com/v1/tracks/2tJulUYLDKOg...,False,True,Grenade,76,https://p.scdn.co/mp3-preview/5221be592c832e14...,1,track,spotify:track:2tJulUYLDKOg9XrtVkMgcJ
1,0.635,0.841,5,-5.379,1,0.0422,0.0134,0.0,0.0622,0.424,...,{'spotify': 'https://open.spotify.com/track/7B...,https://api.spotify.com/v1/tracks/7BqBn9nzAq8s...,False,True,Just the Way You Are,84,https://p.scdn.co/mp3-preview/6d1a901b10c7dc60...,2,track,spotify:track:7BqBn9nzAq8spo5e7cZ0dJ
2,0.709,0.450,9,-8.267,0,0.0428,0.4190,0.0,0.2410,0.873,...,{'spotify': 'https://open.spotify.com/track/5B...,https://api.spotify.com/v1/tracks/5BmKRyAjCnvk...,False,True,Our First Time,54,https://p.scdn.co/mp3-preview/468cf28c47ebf5b7...,3,track,spotify:track:5BmKRyAjCnvkko7Ms00YtA
3,0.658,0.762,10,-3.028,0,0.0425,0.2630,0.0,0.1270,0.823,...,{'spotify': 'https://open.spotify.com/track/5M...,https://api.spotify.com/v1/tracks/5MMLS3xm12D7...,False,True,Runaway Baby,70,https://p.scdn.co/mp3-preview/ee0eec5edbb1a289...,4,track,spotify:track:5MMLS3xm12D7N26xlfFApr
4,0.794,0.711,8,-5.124,0,0.0699,0.3000,0.0,0.0955,0.955,...,{'spotify': 'https://open.spotify.com/track/1E...,https://api.spotify.com/v1/tracks/1ExfPZEiahqh...,False,True,The Lazy Song,72,https://p.scdn.co/mp3-preview/8a6cd6679fc2a388...,5,track,spotify:track:1ExfPZEiahqhLyajhybFeS


In [61]:
uj_merged = pd.merge(uj_features_df, uj_tracks_df,
                     how = 'inner', on = 'id')
uj_merged.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,external_urls,href,is_local,is_playable,name,popularity,preview_url,track_number,type_y,uri_y
0,0.561,0.749,6,-4.951,0,0.0320,0.4610,0.000002,0.2050,0.501,...,{'spotify': 'https://open.spotify.com/track/3G...,https://api.spotify.com/v1/tracks/3G5iN5QBqMeX...,False,True,Young Girls,66,https://p.scdn.co/mp3-preview/1eaad08487defb82...,1,track,spotify:track:3G5iN5QBqMeXx3uZPy8tgB
1,0.726,0.698,5,-4.165,1,0.0431,0.0490,0.000000,0.3090,0.867,...,{'spotify': 'https://open.spotify.com/track/3w...,https://api.spotify.com/v1/tracks/3w3y8KPTfNeO...,False,True,Locked out of Heaven,87,https://p.scdn.co/mp3-preview/5a0318e6c4396478...,2,track,spotify:track:3w3y8KPTfNeOKPiqUTakBh
2,0.717,0.544,7,-6.598,1,0.0371,0.4450,0.000005,0.0973,0.641,...,{'spotify': 'https://open.spotify.com/track/2i...,https://api.spotify.com/v1/tracks/2ih2U8ttFzCj...,False,True,Gorilla,63,https://p.scdn.co/mp3-preview/a42c10dd11306ba9...,3,track,spotify:track:2ih2U8ttFzCjnQ5njF3SrR
3,0.874,0.692,5,-5.280,0,0.0431,0.0412,0.000072,0.3240,0.937,...,{'spotify': 'https://open.spotify.com/track/55...,https://api.spotify.com/v1/tracks/55h7vJchibLd...,False,True,Treasure,81,https://p.scdn.co/mp3-preview/2265c5b16b2e19f6...,4,track,spotify:track:55h7vJchibLdUkxdlX3fK7
4,0.705,0.784,2,-4.351,1,0.0279,0.2110,0.000004,0.0813,0.418,...,{'spotify': 'https://open.spotify.com/track/30...,https://api.spotify.com/v1/tracks/30raivfq7rSt...,False,True,Moonshine,59,https://p.scdn.co/mp3-preview/374f92a8fffde64f...,5,track,spotify:track:30raivfq7rSt5nKltiHfzG


In [62]:
tf_merged = pd.merge(tf_features_df, tf_tracks_df,
                     how = 'inner', on = 'id')
tf_merged.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,external_urls,href,is_local,is_playable,name,popularity,preview_url,track_number,type_y,uri_y
0,0.818,0.803,1,-4.282,1,0.0797,0.0340,0.000000,0.1530,0.632,...,{'spotify': 'https://open.spotify.com/track/6b...,https://api.spotify.com/v1/tracks/6b8Be6ljOzmk...,False,True,24K Magic,82,https://p.scdn.co/mp3-preview/3a76820d510fa5f8...,1,track,spotify:track:6b8Be6ljOzmkOmFslEb23P
1,0.571,0.539,5,-4.446,0,0.1160,0.0208,0.000000,0.1250,0.727,...,{'spotify': 'https://open.spotify.com/track/0m...,https://api.spotify.com/v1/tracks/0mBKv9DkYfQH...,False,True,Chunky,69,https://p.scdn.co/mp3-preview/cb1271bde07433fa...,2,track,spotify:track:0mBKv9DkYfQHjdMcw2jdyI
2,0.853,0.871,8,-3.715,1,0.0851,0.0145,0.000068,0.1980,0.863,...,{'spotify': 'https://open.spotify.com/track/1I...,https://api.spotify.com/v1/tracks/1I6pKIyaBp4O...,False,True,Perm,64,https://p.scdn.co/mp3-preview/0cbb4cd3c7027ca8...,3,track,spotify:track:1I6pKIyaBp4OebTGLJpCCC
3,0.853,0.560,1,-4.961,1,0.0406,0.0130,0.000000,0.0944,0.860,...,{'spotify': 'https://open.spotify.com/track/0K...,https://api.spotify.com/v1/tracks/0KKkJNfGyhkQ...,False,True,That's What I Like,86,https://p.scdn.co/mp3-preview/93046e987d8c5bfd...,4,track,spotify:track:0KKkJNfGyhkQ5aFogxQAPU
4,0.578,0.574,2,-6.209,1,0.0454,0.1960,0.000000,0.0830,0.301,...,{'spotify': 'https://open.spotify.com/track/0k...,https://api.spotify.com/v1/tracks/0kN8xEmgMW9m...,False,True,Versace on the Floor,79,https://p.scdn.co/mp3-preview/6984bfcc42ba35e5...,5,track,spotify:track:0kN8xEmgMW9mh7UmDYHlJP


In [63]:
ss_merged = pd.merge(ss_features_df, ss_tracks_df,
                     how = 'inner', on = 'id')
ss_merged.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,external_urls,href,is_local,is_playable,name,popularity,preview_url,track_number,type_y,uri_y
0,0.696,0.553,1,-8.702,1,0.0759,0.09700,0.000004,0.6170,0.674,...,{'spotify': 'https://open.spotify.com/track/4K...,https://api.spotify.com/v1/tracks/4K09vJ27xCOr...,False,True,Silk Sonic Intro,61,https://p.scdn.co/mp3-preview/652eb965372f40af...,1,track,spotify:track:4K09vJ27xCOreumtSuU6Ao
1,0.586,0.616,5,-7.964,1,0.0324,0.18200,0.000000,0.0927,0.719,...,{'spotify': 'https://open.spotify.com/track/4p...,https://api.spotify.com/v1/tracks/4pryE6cN2gFL...,False,True,Leave The Door Open,70,https://p.scdn.co/mp3-preview/2f06384101ad1a5a...,2,track,spotify:track:4pryE6cN2gFL1FVF5fYINl
2,0.783,0.627,2,-8.521,1,0.1020,0.00756,0.000000,0.0606,0.480,...,{'spotify': 'https://open.spotify.com/track/7s...,https://api.spotify.com/v1/tracks/7suB6D6uKX5D...,False,True,Fly As Me,59,https://p.scdn.co/mp3-preview/e4a561c3113a1cc2...,3,track,spotify:track:7suB6D6uKX5DfPukdGaz0W
3,0.651,0.703,0,-8.958,0,0.0816,0.02970,0.000000,0.0608,0.647,...,{'spotify': 'https://open.spotify.com/track/6j...,https://api.spotify.com/v1/tracks/6jGAh1bFnXt1...,False,True,After Last Night (with Thundercat & Bootsy Col...,66,https://p.scdn.co/mp3-preview/3e85cec2c59de466...,4,track,spotify:track:6jGAh1bFnXt1Muj9zeHveZ
4,0.627,0.618,2,-8.529,1,0.0437,0.05580,0.000000,0.3510,0.848,...,{'spotify': 'https://open.spotify.com/track/1o...,https://api.spotify.com/v1/tracks/1oERlssLrpss...,False,True,Smokin Out The Window,63,https://p.scdn.co/mp3-preview/03985ea0f5b53660...,5,track,spotify:track:1oERlssLrpssCAY6Yqqs6c


In [64]:
album_comp = pd.concat([dw_merged, uj_merged, tf_merged, ss_merged], axis = 0)
album_comp

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,external_urls,href,is_local,is_playable,name,popularity,preview_url,track_number,type_y,uri_y
0,0.704,0.558,2,-7.273,0,0.0542,0.14800,0.000000,0.1070,0.2450,...,{'spotify': 'https://open.spotify.com/track/2t...,https://api.spotify.com/v1/tracks/2tJulUYLDKOg...,False,True,Grenade,76,https://p.scdn.co/mp3-preview/5221be592c832e14...,1,track,spotify:track:2tJulUYLDKOg9XrtVkMgcJ
1,0.635,0.841,5,-5.379,1,0.0422,0.01340,0.000000,0.0622,0.4240,...,{'spotify': 'https://open.spotify.com/track/7B...,https://api.spotify.com/v1/tracks/7BqBn9nzAq8s...,False,True,Just the Way You Are,84,https://p.scdn.co/mp3-preview/6d1a901b10c7dc60...,2,track,spotify:track:7BqBn9nzAq8spo5e7cZ0dJ
2,0.709,0.450,9,-8.267,0,0.0428,0.41900,0.000000,0.2410,0.8730,...,{'spotify': 'https://open.spotify.com/track/5B...,https://api.spotify.com/v1/tracks/5BmKRyAjCnvk...,False,True,Our First Time,54,https://p.scdn.co/mp3-preview/468cf28c47ebf5b7...,3,track,spotify:track:5BmKRyAjCnvkko7Ms00YtA
3,0.658,0.762,10,-3.028,0,0.0425,0.26300,0.000000,0.1270,0.8230,...,{'spotify': 'https://open.spotify.com/track/5M...,https://api.spotify.com/v1/tracks/5MMLS3xm12D7...,False,True,Runaway Baby,70,https://p.scdn.co/mp3-preview/ee0eec5edbb1a289...,4,track,spotify:track:5MMLS3xm12D7N26xlfFApr
4,0.794,0.711,8,-5.124,0,0.0699,0.30000,0.000000,0.0955,0.9550,...,{'spotify': 'https://open.spotify.com/track/1E...,https://api.spotify.com/v1/tracks/1ExfPZEiahqh...,False,True,The Lazy Song,72,https://p.scdn.co/mp3-preview/8a6cd6679fc2a388...,5,track,spotify:track:1ExfPZEiahqhLyajhybFeS
5,0.621,0.820,10,-4.865,1,0.0367,0.33200,0.000000,0.1040,0.4520,...,{'spotify': 'https://open.spotify.com/track/22...,https://api.spotify.com/v1/tracks/22PMfvdz35fF...,False,True,Marry You,74,https://p.scdn.co/mp3-preview/6747a3674ec0b24a...,6,track,spotify:track:22PMfvdz35fFKYnJyMn077
6,0.498,0.590,1,-4.721,0,0.0320,0.51100,0.000000,0.1070,0.0784,...,{'spotify': 'https://open.spotify.com/track/16...,https://api.spotify.com/v1/tracks/161DnLWsx1i3...,False,True,Talking to the Moon,82,https://p.scdn.co/mp3-preview/6feba1b412530ab0...,7,track,spotify:track:161DnLWsx1i3u1JT05lzqU
7,0.691,0.828,1,-3.909,0,0.0749,0.11300,0.000000,0.3580,0.6780,...,{'spotify': 'https://open.spotify.com/track/4K...,https://api.spotify.com/v1/tracks/4KykoaJQ5Uxj...,False,True,Liquor Store Blues (feat. Damian Marley),60,https://p.scdn.co/mp3-preview/34d5bdf562bea2fb...,8,track,spotify:track:4KykoaJQ5UxjCpp3lYkvTE
8,0.589,0.389,0,-5.972,1,0.0280,0.85100,0.000000,0.0772,0.5010,...,{'spotify': 'https://open.spotify.com/track/7l...,https://api.spotify.com/v1/tracks/7l1qvxWjxcKp...,False,True,Count on Me,74,https://p.scdn.co/mp3-preview/525c3c77a8a379c9...,9,track,spotify:track:7l1qvxWjxcKpB9PCtBuTbU
9,0.555,0.814,5,-3.927,0,0.0303,0.09930,0.000004,0.1280,0.5230,...,{'spotify': 'https://open.spotify.com/track/0H...,https://api.spotify.com/v1/tracks/0HasfWMrNxTj...,False,True,The Other Side (feat. CeeLo Green and B.o.B),53,https://p.scdn.co/mp3-preview/e8d0f4c85e1821b1...,10,track,spotify:track:0HasfWMrNxTjycDy9TkRtA


In [66]:
name_pop_eng_temp = album_comp[['name', 'energy', 'popularity', 'tempo']]
                                

In [69]:
name_pop_eng_temp.head(50)

,name,energy,popularity,tempo
0,Grenade,0.558,76,110.444
1,Just the Way You Are,0.841,84,109.021
2,Our First Time,0.450,54,77.986
3,Runaway Baby,0.762,70,163.864
4,The Lazy Song,0.711,72,174.915
5,Marry You,0.820,74,144.905
6,Talking to the Moon,0.590,82,145.867
7,Liquor Store Blues (feat. Damian Marley),0.828,60,75.519
8,Count on Me,0.389,74,89.100
9,The Other Side (feat. CeeLo Green and B.o.B),0.814,53,140.081


# Averages per album (DW, UJ, TF, SS)

## Popularity: 69.9, 68.6, 72, 60.7
## Energy: 0.676, 0.628, 0.690, 0.612
## Tempo: 123.070, 116.269, 136.310, 116.022

### Based on the data provided for the four albums, the average popularity ranges from 60.7 to 72, with 24K Magic being the most popular and An Evening with Silk Sonic the least. The energy levels across the albums are relatively consistent, ranging from 0.612 to 0.690, with 24K Magic having the highest energy and An Evening with Silk Sonic the lowest.

### Tempo varies more significantly, with 24K Magic having the fastest average tempo at 136.310 BPM and An Evening with Silk Sonic the slowest at 116.022 BPM. Overall, 24K Magic stands out for its higher popularity, energy, and tempo compared to the other albums.

### The analysis of the data provides a snapshot of key musical attributes (popularity, energy, and tempo) for the four albums, but it has several limitations. The data is averaged, which masks individual song variation and ignores other factors like lyrical content, genre diversity, or production style, all of which can impact listener reception and popularity. Additionally, the metrics used (energy, tempo) are somewhat abstract and may not fully capture the nuances of the albums.

### Alternative approaches could involve analyzing individual song data for a more granular view, incorporating listener demographics, or using  metrics outside of Spotify's API. Next steps could include examining how these metrics correlate with external factors such as release timing, marketing efforts, or artist collaboration, to better understand what drives an album's overall success and popularity.